# Современные методы распознавания и синтеза речи
## Практическое задание 2: Распознавание речи

В данном задании вы разработаете систему распознавания речи на примере предсказания цифр. Вам будет дана обучающая выборка с произнесеннми цифрами (0-9), а также тестовая выборка аудиофайлов, для которых надо будет предсказать фразы.

**Замечание:** Полный алгоритм описан в книге [Speech and Language Processing](https://1drv.ms/u/s!Ap7BENKQpVbHhxtoTnIqLvI-jo8a), в 9-й главе. При возникновении вопросов, рекомендуем обратиться к этому учебнику.

In [1]:
import warnings
import os
from IPython.display import Audio, display
import numpy as np
import librosa
from librosa.display import specshow
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fftpack import fft, dct, ifft
from sklearn.model_selection import train_test_split
from scipy.stats import norm
warnings.simplefilter('error')
%matplotlib inline

ModuleNotFoundError: No module named 'librosa'

В выданном архиве, помимо Jupter Notebook, находятся следующие файлы:
* cmudict-0.7b.txt – [фонетический словарь CMU (для ознакомления)](https://github.com/Alexir/CMUdict/blob/master/cmudict-0.7b)
* cmu_digits.txt — фонетический словарь цифр
* data/ — аудиофайлы с цифрами. Каждый подкаталог — отдельный класс. В конечных файлах содержится произношение цифр на английском языке.

In [3]:
DATA_PATH = './data/'
SAMPLE_RATE = 16000

In [16]:
# Загружаем данные и делаем train-validation split 
train = dict()
test = dict()
for path in sorted(os.listdir(DATA_PATH)):
    full_path = os.path.join(DATA_PATH, path)
    if os.path.isdir(full_path):
        class_name = path.upper()
        class_files = [os.path.join(full_path, f) for f in sorted(os.listdir(full_path))]
        waves = [librosa.load(file, sr=SAMPLE_RATE)[0] for file in class_files]
        np.random.seed(777) # Не забываем про воспроизводимость
        train_waves, test_waves = train_test_split(waves)
        train[class_name] = train_waves
        test[class_name] = test_waves

In [8]:
display(Audio('data/eight/004ae714_nohash_0.wav'))

In [130]:
# Загружаем словарь фонем (нужен для бонусной части)
phonemes_dictionary = {x[0]: x[1:] for x in [x.split() for x in open('cmu_digits.txt')]}
phonemes_dictionary

{'EIGHT': ['EY1', 'T'],
 'FIVE': ['F', 'AY1', 'V'],
 'FOUR': ['F', 'AO1', 'R'],
 'NINE': ['N', 'AY1', 'N'],
 'ONE': ['W', 'AH1', 'N'],
 'SEVEN': ['S', 'EH1', 'V', 'AH0', 'N'],
 'SIX': ['S', 'IH1', 'K', 'S'],
 'THREE': ['TH', 'R', 'IY1'],
 'TWO': ['T', 'UW1'],
 'ZERO': ['Z', 'IY1', 'R', 'OW0']}

## Часть 1. Извлечение MFCC признаков (опционально, +5 баллов)
Простейшие алгоритмы распознвания речи работают с построенными вручную признаками звуковых сигналов. Обучение на сырых данных, как правило, приводит к низкому качеству распознвания при использовании HMM моделей. В этой части задания мы реализуем функционал для извлечения таких признаков — [MFCC](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum) (Mel frequency cepstrum coefficients).

Построение MFCC коэффициентов происходит в несколько этапов. На первом этапе сигнал разбивается на кадры (frame) при помощи окон. В качестве окна для построения MFCC коэффициентов используется окно Хемминга: 

$$w[n] = \begin{cases}0.54 - 0.46\cos(\frac{2\pi n}{L}) & 0 \le n < L \\ 0 & \textrm{иначе}\end{cases}$$

Параметр $L$ выбирается таким образом, чтобы длина окна соответствовала 25ms. Кадры вычисляются со сдвигом в 10ms, то есть первый кадр будет соответствовать временному интервалу $[0, 25]$, второй — $[10, 35]$ и так далее. Итоговый сигнал получается поэлементным домножением исходного сигнала на сдвинутое окно.

Второй шаг в алгоритме вычисления MFCC — усиление высоких частот (preemphasis). Такой эффект можно добиться, применив фильтр $y[n] = x[n] - \alpha x[n-1]$, где $\alpha \in [0.9, 1]$. Чтобы убедиться, что этот фильтр усиливает высокие частоты, посмотрите на $\mathcal{Z}$-transform этого фильтра.

От каждого полученного окна вычисляется дискретное преобразование Фурье (DFT). Далее сигнал переводится в [mel](https://en.wikipedia.org/wiki/Mel_scale) шкалу, чтобы получить представление, линейное по восприятию. Сопоставление mel и частот (в Гц): $mel(f) = 1127\ln(1+\frac{f}{700})$. Таким образом, частоты ниже 1000Hz преобразуются линейно, а более высокие частоты — логарифмически сжимаются. Перевод осуществляется следующим образом: шкала частот $[0, f_{\max}]$ переводится в mel шкалу: $[0, mel_{\max}]$. Далее mel шкала разбивается на некоторое число равных интервалов. Каждой граничной точке интервала ставится в соответствие частота $f(mel) = (exp(mel/1127)-1)/700$. В результате получаем последовательность частот $f[k]$. Наконец, получаем mel коэффициенты, вычисляя логарифм суммарной энергии в треугольных окнах $f[k-1], f[k], f[k+1]$: $H_k[m] = 
\begin{cases}
0 & m < f[k-1] \\ 
\frac{m-f[k-1]}{f[k]-f[k-1]} & f[k-1] \le m \le f[k] \\
\frac{f[k+1]-m}{f[k+1]-f[k]} & f[k] \le m \le f[k+1] \\
0 & m > f[k+1]\end{cases}$

Mel коэффициенты $i$-ого окна: $S_i[k] = \ln(\sum_{m=0}^{L-1}(|X_i[m]|^2H_k[m])$.

Mel признаки уже можно использовать для обучения моделей распознавания речи. Тем не менее, значительно лучшие результаты получаются при использовании спектра Mel коэффициентов, кепструм. Кепструм — спектр логарифма спектра. Он позволяет отедлить компоненты, создаваемые голосовыми связками и речевым трактом. Формально, для получения кепструма надо взять дискретное косинусное преобразование от mel коэффициентов. Из итоговых коэффициентов обычно оставляют около 12 первых компонент. Дополнительно к этим компонентам добавляют суммарную энергию фрейма. Также добавляют разность признаков соседних фреймов и разность разностей (вторая производная) фреймов. Суммарно получаем 39 коэффициентов на каждый фрейм.

Более детальное описание есть в учебнике и [тут](https://habrahabr.ru/post/140828/).

In [11]:
def extract_mfcc(sound, sampling_rate, shift=32., L=128., mel_coefs=120, mfcc_coefs=12, alpha=0.9, eps=1e-9):
    '''
    Transfroms a sound wave into a sequence of MFCC coefficients.
    :param sound: 1D np.array with sound wave
    :param sampling rate: number of points sampled per second
    :param shift: difference between starting points of consecutive frames, ms.
    :param L: window length, ms.
    Returns 2D array of size (frames x mfcc_dim)
    '''
    # TODO
    return mfcc

In [294]:
# Извлекаем MFCC для всех точек

train_mfccs = dict()
test_mfccs = dict()
for class_name in train:
    mfccs = []
    for sound in train[class_name]:
        mfcc = extract_mfcc(sound, SAMPLE_RATE)
        mfccs.append(mfcc)
    train_mfccs[class_name] = mfccs
    
    mfccs = []
    for sound in test[class_name]:
        mfcc = extract_mfcc(sound, SAMPLE_RATE)
        mfccs.append(mfcc)
    test_mfccs[class_name] = mfccs

Если вы решили не реализовывать MFCC признаки, используйте готовую функцию из `librosa`.

# Часть 2. Обучение ASR модели
Теперь обучим 10 HMM в unsupervised режиме. Каждая будет отвечать за отдельное слово из словаря и оценивать $p(sound | word)$

Если вы застряли с пониманием формул, рекомендуем обратиться к следующей литературе: [Hidden Markov Models](https://web.stanford.edu/~jurafsky/slp3/9.pdf), [HMM cheat sheet](http://www.bcl.hamilton.ie/~barak/misc/hmm.pdf), [A tutorial on HMM](http://www.ece.ucsb.edu/Faculty/Rabiner/ece259/Reprints/tutorial%20on%20hmm%20and%20applications.pdf).

Для инициализации $p(o|i) = \mathcal{N}(\mu_i, \Sigma_i)$ будем использовать глобальную оценку среднего и дисперсии по MFCC признакам: $\mu_i = \mu$, $\Sigma_i = \Sigma$. Также будем рассматривать только диагональные матрицы ковариации.

Альтернативно можно реализовать ASR так, как это было рассказано на лекции (см. [Speech and Language Processing](https://1drv.ms/u/s!Ap7BENKQpVbHhxtoTnIqLvI-jo8a), в 9-й): +10 баллов

In [ ]:
m0 = # TODO
S0 = # TODO

In [570]:
class ASR:
    def __init__(self, phonemes_dictionary, states=10, m0, S0):
        '''
        Initializes HMM
        self.A — statesxstates, array of transition probabilities, initialize randomly
        self.m — statesx39, mean of the p(o|i), initialize with global mean
        self.S — statesx39, variance of the p(o|i), initialize with global variance
        '''
        # TODO
            
    def forward_backward(self, mfcc):
        gamma = # TODO
        xi = # TODO
        return gamma, xi

    def fit(self, train_mfccs, epochs=10):
        '''
        Fits the model using embedded training.
        '''
        # Run forward-backward to train hmm
        # TODO
        
    def get_log_likelihood(self, mfcc):
        # TODO
        return log_likelihood

# Часть 3. Оценка качества модели
Для предсказания слова применим формулу Байеса:
$arg\max p(w|o) = arg\max \frac{p(o|w)p(w)}{p(o)} = arg\max p(o|w)p(w)$. Вероятность $p(w)$ можно оценить по обучающей выборке, а $p(o|w)$ мы уже научились оценивать при помощи HMM. Оцените точность обученной модели по тестовой выборке. 

In [2]:
# TODO

#### Бонус (+5 баллов)
Как зависит качество от числа эпох? От числа состояний сети?

### Оценивание
Суммарные баллы (не более 40):
* Реализация MFCC: +5 баллов
* Обучение обычной HMM: + 15 баллов, или HMM с разделением параметров: + 25 баллов
* Оценка качества модели: +10 баллов
* Параллельная реализация: +5 баллов
* Дополнительные эксперименты по сравнению моделей: +5 баллов